In [1]:
!pip install spotipy
!pip install python-dotenv

from pathlib import Path
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=Path('../.env'))

True

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
# spotify credentials for the application:
# "Related Artist Network Visualizer"
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')
# create a credential manager and api layer
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## 1. Generación de la red.

### Para ello, deberemos empezar por un artista. En este caso empezaremos por J Hook, mi nombre de artista en spotify.

In [3]:
artist_name = "Bad Bunny"

results = sp.search(q='artist:' + artist_name, type='artist')

In [4]:
starting_id = results['artists']['items'][0]['id']

In [7]:
data = sp.artist(starting_id)

data

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4q3ewBCX7sLwd24euuV69X'},
 'followers': {'href': None, 'total': 31368367},
 'genres': ['latin', 'reggaeton', 'trap latino'],
 'href': 'https://api.spotify.com/v1/artists/4q3ewBCX7sLwd24euuV69X',
 'id': '4q3ewBCX7sLwd24euuV69X',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/1c66d291db8ab21ac32d045ef28199fb195b97cf',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/e515d8a5f2a1ec78be486fc51767589ba9790844',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/e64b26e7d388070788f47254c5a439b41b5a76f4',
   'width': 160}],
 'name': 'Bad Bunny',
 'popularity': 99,
 'type': 'artist',
 'uri': 'spotify:artist:4q3ewBCX7sLwd24euuV69X'}

In [10]:
name = data['name']
name

'Bad Bunny'

### Definimos las funciones que, recursivamente (en profundidad), generarán la red de artistas navegando por la API de Spotify.

In [36]:
def related_network(artist_id, depth=3):
    
    graph = dict()
    related_network_recursive(artist_id, depth, graph)
    return graph

def related_network_recursive(artist_id, depth, graph):
    if depth == 0:
        return
    name = sp.artist(artist_id)['name']
    like_artist = sp.artist_related_artists(artist_id)
    graph[name] = [related['name'] for related in like_artist['artists']]
    [related_network_recursive(related['id'], depth - 1, graph) for related in like_artist['artists']]

In [37]:
# comprobamos a profundidad 1
related_network(starting_id, 1)

{'Bad Bunny': ['Lunay',
  'Nio Garcia',
  'Mau y Ricky',
  'Anuel AA',
  'KAROL G',
  'Chris Jedi',
  'Ozuna',
  'Brytiago',
  'Bryant Myers',
  'Rauw Alejandro',
  'Myke Towers',
  'Tainy',
  'Rvssian',
  'Miky Woodz',
  'Lary Over',
  'Danny Ocean',
  'Alex Rose',
  'Jon Z',
  'Kevin Roldan',
  'El Alfa']}

#### Generamos la red. Esta celda tardará mucho tiempo, no solo porque hay muchos artistas que extraer, sino porque necesitamos que cada llamada a la API no se haga de inmediato, lo que provocaría que nos baneen durante unos segundos.

In [38]:
# tomamos la red
depth = 2
network_artist = related_network(starting_id, depth)

#### Guardamos la red para evitar tener que repetir el proceso

In [39]:
import json
name = name.lower().replace(" ", "_")
with open('{}_{}.json'.format(name, depth), 'w') as file:
    json.dump(network_artist, file)

## 2. Análisis de la red

Llegados a este punto, tenemos lista la red para analizarla. Utilizaremos la librería networkx.

In [40]:
import networkx as nx
import numpy as np
import pandas as pd

In [41]:
import json
with open('bad_bunny_2.json', 'r') as file:
    network_artist = json.load(file)

In [42]:
len(network_artist.keys())

21

In [19]:
network_artist['Bad Bunny']

['Lunay',
 'Nio Garcia',
 'Mau y Ricky',
 'Anuel AA',
 'KAROL G',
 'Chris Jedi',
 'Ozuna',
 'Brytiago',
 'Bryant Myers',
 'Rauw Alejandro',
 'Myke Towers',
 'Tainy',
 'Rvssian',
 'Miky Woodz',
 'Lary Over',
 'Danny Ocean',
 'Alex Rose',
 'Jon Z',
 'Kevin Roldan',
 'El Alfa']

Con la función ```edges``` generamos una lista de pares que define las aristas de nuestro grafo.

In [28]:
def edges(graph):
    '''Iterate through the dictionary. 
    For each artist, extend their related artists to create a row for each one.'''
    
    edges = []
    
    [edges.extend(list(zip(np.repeat(artist, len(related_list)), related_list))) 
     for artist, related_list in graph.items()]
    
    return edges

In [29]:
edges_net = edges(network_artist)

In [30]:
edges_net[30:50]

[('Lunay', 'Jory Boy'),
 ('Lunay', 'Kevin Roldan'),
 ('Lunay', 'Anuel AA'),
 ('Lunay', 'Casper Magico'),
 ('Lunay', 'Amenazzy'),
 ('Lunay', 'Tainy'),
 ('Lunay', 'Noriel'),
 ('Lunay', 'Alex Rose'),
 ('Lunay', 'Mau y Ricky'),
 ('Lunay', 'Jhay Cortez'),
 ('Nio Garcia', 'Rvssian'),
 ('Nio Garcia', 'Lunay'),
 ('Nio Garcia', 'Brytiago'),
 ('Nio Garcia', 'Chris Jedi'),
 ('Nio Garcia', 'Lary Over'),
 ('Nio Garcia', 'Bryant Myers'),
 ('Nio Garcia', 'Lalo Ebratt'),
 ('Nio Garcia', 'Myke Towers'),
 ('Nio Garcia', 'DJ Luian'),
 ('Nio Garcia', 'Noriel')]

#### Creamos el grafo utilizando la librería networkx

In [31]:
G = nx.Graph()

In [32]:
G.add_edges_from(edges_net)

In [33]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 73
Number of edges: 346
Average degree:   9.4795


In [34]:
density = nx.density(G)
print("Network density:", density)

Network density: 0.13165905631659056
